In [13]:
import itertools
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import time
import nltk
from nltk.stem import WordNetLemmatizer

import pickle
from gensim import corpora, models, similarities

import psycopg2
import pandas as pd
import numpy as np
from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
SF = pd.read_csv('./data/SF_jobs.csv')
#NYC = pd.read_csv('./data/SF_jobs.csv')
#Seattle = pd.read_csv('./data/SF_jobs.csv')
SF.drop('Unnamed: 0',axis=1,inplace=True)

In [15]:
def text_preprocess(line): # perform tokenization, select noun, Lemmertization etc on a line text

    rtext=[]    
    for w, tag in nltk.pos_tag(nltk.word_tokenize(line.lower())):  # Tokenization
        if tag in ['NN']:  # Keep only Nouns (for this special case)
            rtext.append(w)
    
    wordnet_lemmatizer = WordNetLemmatizer()  # Lemmertization
    rtext = [wordnet_lemmatizer.lemmatize(w) for w in rtext]
            
    stop_words = set(stopwords.words("english")) # Filter out any stop words
    rtext = [w for w in rtext if not w in stop_words]
    
    return rtext

In [24]:
text = []

In [25]:
for i in range (SF.shape[0]):
    text.append(text_preprocess(SF.job_description[i]))

In [36]:
counts = Counter(text)

In [44]:
import operator

frequency = sorted(counts.items(), key=operator.itemgetter(1),reverse=True)


In [46]:
frequency[:1000]

[('experience', 6834),
 ('business', 3225),
 ('development', 2747),
 ('team', 2192),
 ('management', 2162),
 ('knowledge', 1869),
 ('software', 1825),
 ('ability', 1735),
 ('design', 1731),
 ('position', 1411),
 ('work', 1406),
 ('security', 1329),
 ('project', 1321),
 ('technology', 1301),
 ('support', 1218),
 ('information', 1189),
 ('system', 1160),
 ('resume', 1021),
 ('application', 1010),
 ('job', 995),
 ('product', 986),
 ('network', 940),
 ('analysis', 930),
 ('communication', 919),
 ('environment', 900),
 ('understanding', 891),
 ('test', 885),
 ('performance', 877),
 ('engineering', 853),
 ('client', 833),
 ('computer', 819),
 ('time', 793),
 ('company', 789),
 ('quality', 789),
 ('opportunity', 789),
 ('customer', 781),
 ('employment', 781),
 ('engineer', 764),
 ('service', 717),
 ('process', 715),
 ('level', 713),
 ('web', 683),
 ('science', 670),
 ('java', 660),
 ('status', 654),
 ('infrastructure', 653),
 ('role', 649),
 ('platform', 638),
 ('integration', 636),
 ('degree

In [26]:
import pandas as pd
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re 
import pyLDAvis.gensim
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud

In [27]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=20,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [28]:
cleanDescription = text

In [29]:
kagDict   = makeDict(cleanDescription)
kagCorpus = makeCorpus(cleanDescription, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

In [30]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim.prepare(kagLda, kagCorpus, kagDict)
ldaViz

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
12     13.789847        1       1 -0.199827 -0.121020
6      11.613720        1       2 -0.022594 -0.025919
19     11.297710        1       3  0.133287 -0.073604
2       8.522274        1       4  0.040278 -0.005472
1       7.747606        1       5 -0.085647 -0.011542
17      6.590926        1       6  0.075392 -0.051285
18      6.296320        1       7  0.141802 -0.045699
14      5.938530        1       8  0.035483  0.054648
9       5.300698        1       9  0.060421 -0.005069
13      4.647618        1      10  0.069832 -0.201572
10      3.141613        1      11  0.020018  0.056764
16      3.052963        1      12  0.002843  0.050017
15      2.277142        1      13 -0.063596  0.063415
8       2.111749        1      14 -0.142224 -0.064373
5       1.653835        1      15 -0.014210  0.071853
3       1.524359        1      16  0.008293  0.029331
4       1.262222        1      17 -0.015771  0.093004
7       1.191916        1      18  0.108525  0.092472
11      1.167634        1      19 -0.076494 -0.003117
0       0.871317        1      20 -0.075812  0.097167, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
1554   Default  6473.000000   experience  6473.000000  30.0000  30.0000
3537   Default  1246.000000     security  1246.000000  29.0000  29.0000
3182   Default   885.000000      network   885.000000  28.0000  28.0000
211    Default  2591.000000  development  2591.000000  27.0000  27.0000
2847   Default  1752.000000     software  1752.000000  26.0000  26.0000
2276   Default  3128.000000     business  3128.000000  25.0000  25.0000
11182  Default   330.000000     deloitte   330.000000  24.0000  24.0000
4773   Default  1337.000000     position  1337.000000  23.0000  23.0000
645    Default  1139.000000      support  1139.000000  22.0000  22.0000
309    Default  1319.000000      project  1319.000000  21.0000  21.0000
6162   Default   976.000000       resume   976.000000  20.0000  20.0000
8420   Default  1747.000000    knowledge  1747.000000  19.0000  19.0000
10299  Default   476.000000  eligibility   476.000000  18.0000  18.0000
1625   Default  1117.000000  information  1117.000000  17.0000  17.0000
5459   Default  2075.000000   management  2075.000000  16.0000  16.0000
3643   Default   850.000000         test   850.000000  15.0000  15.0000
4972   Default   539.000000     database   539.000000  14.0000  14.0000
11522  Default   752.000000   employment   752.000000  13.0000  13.0000
10758  Default   876.000000     analysis   876.000000  12.0000  12.0000
3163   Default   289.000000         bank   289.000000  11.0000  11.0000
10281  Default  1634.000000       design  1634.000000  10.0000  10.0000
8776   Default   318.000000         risk   318.000000   9.0000   9.0000
3808   Default   942.000000      product   942.000000   8.0000   8.0000
11350  Default   930.000000          job   930.000000   7.0000   7.0000
2344   Default   733.000000     customer   733.000000   6.0000   6.0000
5455   Default   745.000000      quality   745.000000   5.0000   5.0000
1228   Default  1225.000000   technology  1225.000000   4.0000   4.0000
2489   Default  1097.000000       system  1097.000000   3.0000   3.0000
10677  Default   641.000000         java   641.000000   2.0000   2.0000
1589   Default   739.000000      company   739.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
2847   Topic20    22.148479     software  1752.778582   0.3717  -4.2815
1353   Topic20    13.152935     industry   516.916531   1.0717  -4.8026
6931   Topic20     5.634854       mining    65.779304   2.2856  -5.6503
1228   Topic20    15.490543   technology  1225.018057   0.3724  -4.6391
11350  Topic20    12.526799          job   930.927109   0.4346  -4.8514
2215   Topic20     6.285425    al